In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('2018CrimeStatisticsByCategoryOfCrime.csv')
df

In [ ]:
# Drop null values
df.dropna(inplace=True)
df

In [ ]:
df.columns

In [ ]:
# Check for null values
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Checking null values in figures
import missingno as msno
msno.bar(df, color='gray', fontsize=24);

In [ ]:
"""
- Key Objectives for analyzing your dataset
- Features of the columns you analyzing and why are you analyzing.
- Processing and Preparation | Why you chose them
- 
"""

In [ ]:
# By Martin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# load the uce results dataset
df = pd.read_csv('UceResultsBySchool2011-2016.csv')
df

In [ ]:
# Dataset before cleaning 20
df.head(20)

In [ ]:
# Check for shape | helps to determine No of rows and columns
df.shape

In [ ]:
# Preparation and Preprocessing or Data cleaning
df.columns

In [ ]:
# Removing white spaces
df.columns = df.columns.to_series().apply(lambda x: x.strip())
df.columns

In [ ]:
# Repalce column names with underscores
df.columns = df.columns.str.replace(" ", "_")
df.columns

In [ ]:
# Check for info
df.info()

In [ ]:
# Check for duplicates and drop them
for isduplicate in df.duplicated().array:
    if isduplicate:
        print('uplicates identified and dropped')
        df.drop_duplicates(inplace=True)
        break
    else:
        pass
print('No duplicates found')
df[df.duplicated()==True]

In [ ]:
# Check for null values or null
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Check for null and Nan values in the dataset
df.isnull().any()

In [ ]:
# Check for the Number of rows that are null
null_rows = df[df.isnull().any(axis=1)]
before_computation = len(null_rows.axes[0])
print('Number of rows that are null values: ', before_computation)

In [ ]:
# Remove rows without school names
df.dropna(thresh=None, subset=['SCHOOL'], inplace=True)

In [ ]:
# Replace missing TOTAL CANDIDATES with sum of MAlE and FEMALE candidates since non of the two has a null value
df['TOTAL_CANDIDATES'].fillna(df['MALE_CANDIDATES']+df['FEMALE_CANDIDATES'], inplace=True)

In [ ]:
# Now we need to check if the total sum of male and female is equal to total candidates
notEqual = df[df['TOTAL_CANDIDATES'] != (df['MALE_CANDIDATES']+df['FEMALE_CANDIDATES'])]
notEqual[['SCHOOL', 'TOTAL_CANDIDATES', 'MALE_CANDIDATES', 'FEMALE_CANDIDATES']]

In [ ]:
# generating grades array [1,2,3,4,7,9,x] from each percentage grade
grades = []
for column in df.columns:
    if(column.startswith("%")):
        grades.append(column[-1])

# Getting coefficient of key Sections
for row in df.index:
    total_coefficient = df.at[row, "TOTAL_CANDIDATES"] / 100 
    male_coefficient = df.at[row, "MALE_CANDIDATES"] / 100 
    female_coefficient = df.at[row, "FEMALE_CANDIDATES"] / 100 

    # for each grade [1,2,3,4,7,9,x] obtain it's total,%,maleTotal,male%,femaleTotal,female%
    for grade in grades:
        total = list(filter(lambda x: x.endswith(grade) and x.startswith("TOTAL"), df.columns))[0]
        totalPercentage = list(filter(lambda x: x.endswith(grade) and x.startswith("%"), df.columns))[0]
        maleTotal = list(filter(lambda x: x.endswith(grade) and x.startswith("MALE") and "TOTAL" in x, df.columns))[0]
        malePercentage = list(filter(lambda x: x.endswith(grade) and x.startswith("MALE") and "%" in x, df.columns))[0]
        femaleTotal = list(filter(lambda x: x.endswith(grade) and x.startswith("FEMALE") and "TOTAL" in x, df.columns))[0]
        femalePercentage = list(filter(lambda x: x.endswith(grade) and x.startswith("FEMALE") and "%" in x, df.columns))[0]

        # Here whenever the coefficient is 0, fill the remaining related fields for that grade with 0's 
        if(total_coefficient == 0.0):
            df.at[row, total] = 0.0
            df.at[row, totalPercentage] = 0.0
        if(total_coefficient == 0.0):
            df.at[row, maleTotal] = 0.0
            df.at[row, malePercentage] = 0.0
        if(total_coefficient == 0.0):
            df.at[row, femaleTotal] = 0.0
            df.at[row, femalePercentage] = 0.0

        # whenever percentage grade 0.0% replace its corresponding NaN grade with 0.0
        if(df.at[row, totalPercentage] == 0.0 and pd.isna(df.at[row, total])):
            df.at[row, total] == 0.0
        if(df.at[row, femalePercentage] == 0.0 and pd.isna(df.at[row, femaleTotal])):
            df.at[row, femaleTotal] == 0.0
        if(df.at[row, malePercentage] == 0.0 and pd.isna(df.at[row, maleTotal])):
            df.at[row, maleTotal] == 0.0

        # whenever grade 0.0 replace its corresponding NaN %grade with 0.0
        if(df.at[row, total] == 0.0 and pd.isna(df.at[row, totalPercentage])):
            df.at[row, totalPercentage] == 0.0

        if(df.at[row, femaleTotal] == 0.0 and pd.isna(df.at[row, femalePercentage])):
            df.at[row, femalePercentage] == 0.0

        if(df.at[row, maleTotal] == 0.0 and pd.isna(df.at[row, malePercentage])):
            df.at[row, malePercentage] == 0.0

        # using the calculated coefficients
        if(pd.notna(df.at[row, total]) and pd.isna(df.at[row, totalPercentage])):
            df.at[row, totalPercentage] = round((df.at[row, total] / total_coefficient), 1)

        if(pd.notna(df.at[row, totalPercentage]) and pd.isna(df.at[row, total])):
            df.at[row, total] = int(df.at[row, totalPercentage] * total_coefficient)

        if(pd.notna(df.at[row, maleTotal]) and pd.isna(df.at[row, malePercentage])):
            df.at[row, malePercentage] = round((df.at[row, maleTotal] / male_coefficient), 1)

        if(pd.notna(df.at[row, malePercentage]) and pd.isna(df.at[row, maleTotal])):
            df.at[row, maleTotal] = int(df.at[row, malePercentage] * male_coefficient)

        if(pd.notna(df.at[row, femaleTotal]) and pd.isna(df.at[row, femalePercentage])):
            df.at[row, femalePercentage] = round((df.at[row, femaleTotal] / female_coefficient), 1)

        if(pd.notna(df.at[row, femalePercentage]) and pd.isna(df.at[row, femaleTotal])):
            df.at[row, femaleTotal] = int(df.at[row, femalePercentage] * female_coefficient)

        # applying subtraction from Total %ages, to fill the missing ones ie male and female percentages
        if(pd.notna(df.at[row, malePercentage]) or pd.notna(df.at[row, femalePercentage]) is not pd.notna(df.at[row, femalePercentage]) == pd.notna(df.at[row, malePercentage])):
            if(pd.notna(df.at[row, malePercentage])):
                df.at[row, femaleTotal] = df.at[row, totalPercentage] - df.at[row, malePercentage]

            if(pd.notna(df.at[row, femalePercentage])):
                df.at[row, maleTotal] = df.at[row, totalPercentage] - df.at[row, femalePercentage]


        # get missing total grades by summation and subtraction of male, female and general totals
        if not (pd.notna(df.at[row, total]) and pd.notna(df.at[row, maleTotal]) and pd.notna(df.at[row, femaleTotal])):
            if(pd.notna(df.at[row, maleTotal]) and pd.notna(df.at[row, total])):
                df.at[row, femaleTotal] = df.at[row, total] - df.at[row, maleTotal]
            if(pd.notna(df.at[row, femaleTotal]) and pd.notna(df.at[row, total])):
                df.at[row, maleTotal] = df.at[row, total] - df.at[row, femaleTotal]
            if(pd.notna(df.at[row, maleTotal]) and pd.notna(df.at[row, femaleTotal])):
                df.at[row, total] = df.at[row, femaleTotal] + df.at[row, maleTotal]



In [ ]:
print('Number of rows with null values BEFORE running the algorithm: ', before_computation)
withnullrows_after = df[df.isna().any(axis=1)]

print("Number of rows with null values AFTER running the algorithm: "+str(len(withnullrows_after.axes[0])))

In [ ]:
df.isna().any()

In [ ]:
df.SCHOOL = df.SCHOOL.apply(lambda x: x.split(" ")[0].strip())


for school in sorted(df.SCHOOL.unique()):
    print(school)

In [ ]:
yearCandidates = df[['YEAR', 'TOTAL CANDIDATES']]
yearCandidates

In [ ]:
total_year_candidates = yearCandidates.groupby('YEAR').sum()
total_year_candidates

In [ ]:
YearCandidatesBarh = total_year_candidates.plot(kind='barh', align='center', figsize=(10, 3), color = 'indigo', legend=False,
                                                title="HORIZONTAL BAR GRAPH SHOWING TOTAL CANDIDATES WHO SIT UCE PER YEAR",
                                                ).set_xlabel('TOTAL_CANDIDATES')
print('This shows that year 2016 had the highest number of UCE candidates: ')
YearCandidatesBarh

In [ ]:
df.columns

In [ ]:
yearGrades = df[['YEAR', 'TOTAL DIV 1', 'TOTAL DIV 2', 'TOTAL DIV 3', 'TOTAL DIV 4', 'TOTAL DIV 7', 'TOTAL DIV 9', 'TOTAL X']]
yearGrades

In [ ]:
total_year_grades = yearGrades.groupby('YEAR')[['TOTAL DIV 1', 'TOTAL DIV 2', 'TOTAL DIV 3', 'TOTAL DIV 4', 'TOTAL DIV 7', 'TOTAL DIV 9', 'TOTAL X']]
total_year_grades

In [ ]:
# plotting total candidates in each division per year
total_year_grades.plot(kind='bar', align='center', figsize=(15, 10), 
                       title="BAR GRAPH SHOWING TOTAL UCE CANDIDATES IN GIVEN DIVISIONS PER YEAR").tick_params(axis='x', labelrotation=0)